In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation # animaciones
from IPython.display import HTML

from scipy.sparse import diags, kron, csr_matrix, csc_matrix, identity, eye
from scipy.sparse.linalg import spsolve, norm, inv, eigsh

import time

from scipy.fft import fft, ifft, dct, idct
from scipy.integrate import odeint, solve_ivp

# Colisiones Kink-Antikink en ecuación $\phi_4$

En el presente jupyter se simulará una colisión de ondas kink-antikink en la ecuación $\phi_4$ dada por
$$
\begin{equation}
\left.
\begin{array}[c]{rll}
 \phi_{tt} - \phi_{xx} - \phi + \phi^3= & 0 & \text{sobre }\Omega=\left(0,T\right]\times\left(-L,L\right)\\
\phi(0,x)= & \phi_K(x+x_0) + \phi_{\bar{K}}(x-x_0)-1 & \text{en }x\in [-L, L]\\
\phi_t(0,x)= & -v\phi'_K(x+x_0) - v\phi'_{\bar{K}}(x-x_0) & \text{en }x\in [-L, L]
\end{array}
\right\} \quad (1)
\end{equation}
$$

Con condiciones de borde periódica o Neumann según lo que se requiera. Los objetivos son:
* Encontrar el mejor método de simulación, es decir, aquel método con menor propagación de errores.
* Obtener la velocidad de salida después de la colisión.
* Obtener los gráficos correspondientes, de forma de replicar los resultados del paper.

## introducción

Los métodos utilizados para simular la ecuación corresponden a diferencias finitas (esquemas explícito e implícito) y métodos espectrales. Es esperable que la presición de la simulación mejor con respecto a cada método usado, donde los esquemas espectrales tengan menor dispersión de error y deseablemente deban ser utilizados en la obtención de las velocidades de salida.

Se definen en primer lugar funciones con las condiciones iniciales del problema:

In [2]:
# Función secante hiperbolica
def sech(x):
    return 1/np.cosh(x)

# Función kink
def phi_k(x, v):
    return np.tanh(x/(np.sqrt(2) * np.sqrt(1-v**2)))

# Derivada kink
def phi_k_t(x, v):
    return sech(x/(np.sqrt(2) * np.sqrt(1-v**2)))**2

# Condición inicial
def phi_0(x, x0, v):
    return phi_k(x+x0, v) - phi_k(x-x0, v)-1

# Condición inicial derivada temporal
def phi_1(x, x0, v):
    return -v/np.sqrt(2*(1-v**2)) * (phi_k_t(x+x0, v) + phi_k_t(x-x0, v))

En todas las simulaciones, se utilizarán $x_0 = 10$, $L = 20$ y tiempo final $T=100$. Este último parámetro podría variar según la velocidad de cálculo computacional de cada esquema.

In [3]:
x0 = 10
xi = -20
xf = 20
tf = 100

A modo de testeo, se probarán las velocidades $v \in \{0.35, 0.186, 0.1988, 0.213\}$.

In [4]:
# Velocidades
vs = np.arange(0.19, 0.2, 0.001) #[0.189, 0.1988, 0.2384, 0.35]

## Diferencias finitas: Esquema explícito

Para $h>0$ y $dt = h/2$ (se debe cumplir la condición CFL) se obtienen una grilla espacial de $N$ puntos y una temporal de $N_T = T/dt$ puntos, además del siguiente problema discretizado centralizado en tiempo y espacio:

$$
\frac{\phi_j^{n+1} - 2\phi_j^n + \phi_j^{n-1}}{dt^2} - \frac{\phi_{j+1}^n - 2\phi_j^n + \phi_{j-1}^n}{h^2} + f(\phi_j^n) = 0
$$

donde $f(\phi) = -\phi + \phi^3$. Definimos $\lambda = \frac{dt}{h}$ y despejando $\phi_j^{n+1}$ se obtiene un esquema explícito para la ecuación $\phi_4$:

$$
\phi_j^{n+1} = 2\left (1-\lambda^2\right )\phi_j^n - \phi_j^{n-1} + \lambda^2\phi_{j+1}^n + \lambda^2\phi_{j-1}^n - dt^2f(\phi_j^n)
$$
Con condiciones iniciales:

* $\phi_j^0 = \phi_K(x_j+x_0) + \phi_{\bar{K}}(x_j-x_0)-1$
* $\phi_j^1 = \left(-v\phi'_K(x_j+x_0) - v\phi'_{\bar{K}}(x_j-x_0)\right) dt + \phi_j^0$

y con condiciones de borde periódicas (que será resuelto utilizando el método ``np.roll``). Este esquema se implementa a continuación en la función ``solve_explicit``.

In [5]:
def solve_explicit(x0, xi, xf, h, dt, tf, v):
    
    # Puntos de las grillas
    Nx = int((xf - xi)/h)
    Nt = int(tf/dt)

    # Crea grillas
    x = np.arange(xi,xf, h)
    t = np.arange(0, tf, dt)

    # Vector para guardar solución
    phi = np.zeros((Nt, Nx))

    # Setea condiciones iniciales
    phi[0] = phi_0(x, x0, v)
    phi[1] = phi_1(x, x0, v) * dt + phi[0]
    
    # Lambda
    l = dt/h
    
    # Itera puntos temporales
    for n in range(2, Nt):
        
        # Instante anterior y paso actual
        phi_j_last = phi[n-2]
        phi_j = phi[n-1]

        # Condiciones periódicas
        
        # Vecino derecha
        phi_j_der = np.roll(phi_j, shift=1)

        # Vecino izquierda
        phi_j_izq = np.roll(phi_j, shift=-1)

        # Solución instante actual
        phi_j_sgte =  2 * (1 - l**2) * phi_j - phi_j_last + l**2 * (phi_j_der + phi_j_izq) + dt**2 * (phi_j - phi_j**3)

        # Guarda solución
        phi[n] = phi_j_sgte
    
    # Retorna grillas y solución
    return x, t, phi

Se simulará el modelo para las velocidades antes mencionadas y se observará el comportamiento de la solución en cada una. Además, se utilizará $h=0.1$.

In [6]:
# Parámetros de discretización
h = 0.1
dt = h/2

phis = []

for v in vs:
    x, t, phi = solve_explicit(x0, xi, xf, h, dt, tf, v)
    phis += [phi]

In [7]:
def get_plot_phi(x, t, phi, v, doc_name):
    X, T = np.meshgrid(x, t)
    
    plt.title(r"Solución $\phi^4$ para $v=${}".format(v))
    plt.pcolormesh(T, X, phi, shading="auto")
    
    plt.colorbar()
    plt.xlabel("t")
    plt.ylabel("x")
    
    #plt.savefig(doc_name)
    plt.show()

In [8]:
# Directorio gifs esquema explicito
path_exp = "Chars/Explicit"

#for i in range(len(phis)):
    #get_plot_phi(x, t, phis[i], vs[i], path_exp+"/calor_v_{}".format(i))

Experimentando con distintas tamaños de grillas espaciales, se determinó que a partir de $h=0.1$ no ocurren cambios notorios en la solución. Por otro lado, se comprobó que $dt=\frac{h}{2}$ resulta ser también un buen espaciado temporal.

Como añadido extra, se define una función que crea animaciones de la solución. Esto ayudará a obtener las velocidades de salida de manera más ilustrativa.

In [9]:
def animar_soluciones(x, phi, v, dt, tf, doc_name, path):
    
    # Cantidad de frames
    Nt = int(tf/dt)

    # Crea figura
    fig, ax = plt.subplots()
        
    # Grafica instante inicial
    line1, = ax.plot(x, phi[0])
    #line1, = ax.plot(x, (phi[0] + 1)*np.cosh(x))
    #line2, = ax.plot(x, -1 + sech(x))
    ax.set_ylim(-2, 2)
    ax.set_title("Solución $\phi^4$ para $v=${}".format(v))

    # Funcion para animar
    def animate(i):
        
        # Actualiza grafico en cada instante temporal
        line1.set_data(x, phi[i])

        return line1,

    # Animación
    ani = animation.FuncAnimation(fig, animate, frames=np.arange(0, Nt, 10), interval=50, blit=True)

    ani.save(path + "/" + doc_name)
    return ani

In [10]:
#for i in range(len(phis)):
    #animar_soluciones(x, phis[i], vs[i], dt, tf, "Exp_v_{}.gif".format(i), path_exp)
    
# Deprecado
#animar_soluciones(x, phi_second, Nt, "Caso_2_phi_4.gif", path)
#animar_soluciones(x, phi_third, Nt, "Caso_3_phi_4.gif", path)

#animar_soluciones(x, phi_second, Nt, "Caso_2_phi_4_sech.gif", path)

## Esquema implícito

Dada la dificultad al invertir matrices en problemas con condiciones de border periódicas, se utilizan condiciones neumann nulas en los bordes.

In [11]:
def f(u):
    return -u + u**3

def Ah(l, N):
    
    e = np.ones(N)*(1 + l ** 2)
    e[0] = e[-1] = 1 + (l**2)/2 # En los extremos vale 1+lambda^2/2
    
    f = np.ones(N-1)*(l**2)/2
    
     # Arreglo con las diagonales
    k = np.array([-f,e,-f], dtype= object) 
    offset = [-1,0,1] #Posiciones de las diagonales
    
    A = diags(k, offset, format="csc")
    return A

def bn(u, dt):
    return 2 * u - f(u) * dt**2

def solve_implicit(x0, xi, xf, h, dt, tf, v, return_bounces=False):
    
    # Puntos de cada grilla
    Nx = int((xf - xi)/h)
    Nt = int(tf/dt)
    
    # Grillas
    x = np.linspace(xi,xf, Nx)
    t = np.linspace(0, tf, Nt)
    
    # Lambda
    l = dt / h

    u = np.zeros((Nt, Nx))

    # Setea condiciones iniciales
    u[0] = phi_0(x, x0, v)
    u[1] = phi_1(x, x0, v) * dt + u[0]
    
    # Obtiene matriz A
    A = Ah(l, Nx)
    
    # Guarda n bounces
    n_bounces = []
    
    # Recorre puntos temporales
    for n in range(1, Nt-1):
        
        # Paso actual y anterior
        un = u[n]
        un_last = u[n-1]
        
        # Vector b
        b = bn(un, dt)
        
        # Invierte matriz
        un_sig = spsolve(A, b)
        
        # Actualiza paso siguiente
        u[n+1] = un_sig - un_last # Habiamos resuelto para u_{n+1} + u_{n-1}
        
        if np.min(u[n+1]) < -1.5:
            n_bounces += [n+1]
    
    if return_bounces:
        return x, t, u, n_bounces
    
    # Retorna grillas y solucion
    return x, t, u

### Simulación

In [12]:
# Parámetros de discretización
h = 0.01
dt = 0.01

# Guarda soluciones y grillas espaciales
us = []
xs = []

# Guarda lista de veces que la solución fue menor a -1.5
bounces = np.full(len(vs), fill_value=None)

for i in range(len(vs)):
    v = vs[i]
    x, t, u, bounce = solve_implicit(x0, xi, xf, h, dt, tf, v, return_bounces=True)
    us += [u]
    xs += [x]
    bounces[i] = bounce
    

In [13]:
# Obtiene animaciones para cada velocidad
#for i in range(len(us)):
    #animar_soluciones(xs[i], us[i], vs[i], dt, tf, "Imp_v_{}.gif".format(i), path_imp)

In [14]:
# Directorio gifs esquema explicito
path_imp = "Charts/Implicit"

#for i in range(len(phis)):
#    get_plot_phi(x, t, us[i], vs[i], path_imp+"/calor_v_{}".format(i))

## Método espectral

Se utilizará el método de colocación, que corresponde a un método pseudoespectral y donde la solución se aproxima evaluando en puntos de colocación estratégicos, o mejor dicho, una malla discreta de puntos. Primeramente se usarán las frecuencias de sampleos de la transformada de Fourier, que se obtienen con el método ``np.fft.fftfreq`` y luego una discretización de malla útil para la transformada del coseno.

La primera sección utilizará una combinación entre las frecuencias de Fourier y la transformada del coseno. La segunda sección será destinada totalmente a la transformada del coseno, puesto que la condición inicial del problema es par y cuya paridad se preserva en la ecuación.

### Híbrido coseno y frecuencias de Fourier

Para resolver las EDO´s que aparezcan, se utilizará Runge Kutta 4. La implementación incluye un parámetro para retornar los bounces de la solución. Esta variante permite calcular la velocidad de salida de la solución.

In [16]:
def RK4(f, ti, tf, h, X0, args, get_bounces=False):
    """ Función que entrega la aproximación numérica de Runge Kutta 4 
    para la solución al problema X' = f(t,X).

    f: Función f(t,X,args) que es función de lado derecho de la ecuación
    ti: t inicial
    tf: t final
    h: Paso h del método
    X0: Condición inicial
    args: Argumentos a entregar a la función f

    Regresa (t,X) donde t es el intervalo [ti,tf] equiespaciado y X
    la aproximación a la solución mediante método de Runge Kutta de orden 4.
    """
    # Cardinal del equiespaciado del método
    N = int((abs(tf-ti))/h)

    # Arreglo de t
    t = np.linspace(ti, tf, N)
    
    Nx = len(X0)

    # Arreglo para la solución
    X = np.zeros((N, Nx))

    # Condición inicial
    X[0] = X0
    
    # Guarda n bounces
    n_bounces = []

    # Método de Runge Kutta 4
    for i in range(N-1):
        g1 = f(t[i]      , X[i]       , args)
        g2 = f(t[i] + h/2, X[i]+h/2*g1, args)
        g3 = f(t[i] + h/2, X[i]+h/2*g2, args)
        g4 = f(t[i] + h  , X[i]+h*g3  , args)
        X[i+1] = X[i] + h/6*(g1 + 2*g2 + 2*g3 + g4)
        
        
        # Obtiene bounces de la solución
        if get_bounces:
            X_bounces = X[:, :Nx//2]
        
            if np.min(X_bounces[i+1]) < -1.5:
                n_bounces += [i+1]

    if get_bounces:
        return t, X.T, n_bounces
    return (t,X.T)

Definimos los parámetros de discretización:

In [17]:
L = 20
dt = 0.1
tend = 120
x0 = 10
nx = 128

Se define la función ``rhsf`` que retorna la transformada de coseno aplicada a la ecuación $\phi_4$ y evaluada en los puntos de colocación para la transformada de Fourier.

In [18]:
def rhsf(phi, t, L):
    nx = len(phi) #Podria tener error
    n = np.fft.fftfreq(nx) * nx * 2 * np.pi / L
    return idct((2 * np.pi * 1j * n / L)**2 * dct(phi)).real + phi - phi**3

Definimos las derivadas del sistema. Como tenemos dos derivadas en tiempo, se define $z = \phi_t$, obteniendo dos edos por cada valor arrojado en la función ``rhsf``.

In [19]:
def system_derivative(t, y, L):
    total = len(y)
    phi = y[:total//2].copy()
    z = y[total//2:].copy()

    dphi = z
    dz = rhsf(phi, t, L)

    dy = np.append(dphi, dz)

    return dy

Definimos una función que nos arroje la solución con el método mencionado. Recibirá los parámetros antes definidos y retornará la solución deseada.

In [20]:
def solve_fft(L, dt, tend, x0, nx, v):
    dx = L/nx

    # Grilla
    x = np.linspace(-L, L, nx)

    # Condiciones iniciales
    phi0 = phi_0(x, x0, v)
    phi1 = phi_1(x, x0, v)
    y0 = np.append(phi0, phi1)

    t, y, bounces = RK4(system_derivative,0, tend, dt, y0, L, get_bounces=True)
    
    # Rescata la solución que nos interesa
    total = len(y)
    y_sol = y[:total//2]
    y_sol = y_sol.T
    
    return t, x, y_sol, bounces